In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))


import random

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch
from tensorflow import keras

tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from notebooks.helper_functions import (
    convert_bytes,
    convert_prefetchdataset_to_numpy_arrays,
    evaluate_time_of_prediction,
    get_file_size,
    predict_and_print_full_results,
)

2024-09-22 17:45:18.922817: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-22 17:45:18.965797: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-22 17:45:18.965828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-22 17:45:18.968292: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-22 17:45:18.977777: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-22 17:45:18.978514: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [4]:
train_dataset = tf.keras.utils.audio_dataset_from_directory(
    "../../dataset/training",
    labels="inferred",
    sampling_rate=16000,
    batch_size=32,
    shuffle=True,
    seed=3407,
)
test_dataset = tf.keras.utils.audio_dataset_from_directory(
    "../../dataset/testing",
    labels="inferred",
    sampling_rate=16000,
    batch_size=32,
    shuffle=True,
    seed=3407,
)
val_dataset = tf.keras.utils.audio_dataset_from_directory(
    "../../dataset/validation",
    labels="inferred",
    sampling_rate=16000,
    batch_size=32,
    shuffle=True,
    seed=3407,
)

label_names = np.array(train_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-09-22 17:45:36.087331: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-22 17:45:36.087784: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-09-22 17:45:36.161771: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-09-22 17:45:36.168977: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [5]:
x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(
    train_dataset, data_type="time-series"
)
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(
    val_dataset, data_type="time-series"
)
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(
    test_dataset, data_type="time-series"
)

### HPO

In [5]:
class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes, num_of_layers):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.num_of_conv_layers = num_of_layers

    def build(self, hp):
        model = tf.keras.Sequential()

        # Hyperparameters
        kernel_size = hp.Choice("kernel_size", values=[2, 3, 5])
        dense_units = hp.Int("1st_dense_units", min_value=4, max_value=32, step=4)
        dense_activation = hp.Choice(
            "2nd_dense_activation", values=["softmax", "sigmoid"]
        )
        learning_rate = hp.Float(
            "learning_rate", min_value=1e-4, max_value=1e-2, sampling="log"
        )

        # Model architecture
        model.add(tf.keras.layers.Input(shape=self.input_shape))

        # Tune number of layers
        for i in range(self.num_of_conv_layers):
            model.add(
                tf.keras.layers.Conv1D(
                    filters=hp.Int(f"filters_{i}", min_value=2, max_value=8, step=2),
                    kernel_size=kernel_size,
                    activation=hp.Choice("conv2d_activation", ["relu", "tanh"]),
                )
            )
            model.add(
                tf.keras.layers.MaxPooling1D(
                    hp.Int("pull_size", min_value=4, max_value=10, step=2)
                )
            )

        model.add(tf.keras.layers.Flatten())

        model.add(tf.keras.layers.Dense(units=dense_units, activation="relu"))

        model.add(
            tf.keras.layers.Dense(units=self.num_classes, activation=dense_activation)
        )

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=["accuracy"],
        )
        return model

#### Tune 1 conv2d layer CNN

In [6]:
input_shape = (48000, 1)
num_classes = 2

num_of_conv_layers = 1
tuner_1_layer_cnn = RandomSearch(
    CNNHyperModel(input_shape, num_classes, num_of_conv_layers),
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=2,
    directory="hpo_tuner/cnn",
    project_name="cnn_time_series_1_conv1d_tuning",
)
tuner_1_layer_cnn.search(train_dataset, epochs=5, validation_data=val_dataset)
tuner_1_layer_cnn.results_summary()

Trial 10 Complete [00h 07m 29s]
val_accuracy: 0.9322463870048523

Best val_accuracy So Far: 0.9615941941738129
Total elapsed time: 01h 14m 15s
Results summary
Results in hpo_tuner/cnn/cnn_time_series_1_conv1d_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 07 summary
Hyperparameters:
kernel_size: 5
1st_dense_units: 20
2nd_dense_activation: sigmoid
learning_rate: 0.00392551860940815
filters_0: 4
conv2d_activation: relu
pull_size: 8
Score: 0.9615941941738129

Trial 06 summary
Hyperparameters:
kernel_size: 5
1st_dense_units: 16
2nd_dense_activation: sigmoid
learning_rate: 0.0018228072765774177
filters_0: 8
conv2d_activation: relu
pull_size: 6
Score: 0.9550724625587463

Trial 05 summary
Hyperparameters:
kernel_size: 5
1st_dense_units: 28
2nd_dense_activation: softmax
learning_rate: 0.00458502351256499
filters_0: 4
conv2d_activation: tanh
pull_size: 4
Score: 0.9503623247146606

Trial 08 summary
Hyperparameters:
kernel_size: 5
1st_dense_units: 28
2nd_dens

#### Tune 2 conv2d layers CNN

In [7]:
num_of_layers = 2
tuner_2_layer_cnn = RandomSearch(
    CNNHyperModel(input_shape, num_classes, num_of_layers),
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=2,
    directory="hpo_tuner/cnn",
    project_name="cnn_time_series_2_conv1d_tuning",
)
tuner_2_layer_cnn.search(train_dataset, epochs=5, validation_data=val_dataset)
tuner_2_layer_cnn.results_summary()

Trial 10 Complete [00h 07m 25s]
val_accuracy: 0.8014492690563202

Best val_accuracy So Far: 0.969565212726593
Total elapsed time: 01h 10m 33s
Results summary
Results in hpo_tuner/cnn/cnn_time_series_2_conv1d_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 02 summary
Hyperparameters:
kernel_size: 2
1st_dense_units: 16
2nd_dense_activation: softmax
learning_rate: 0.008792598590199204
filters_0: 4
conv2d_activation: tanh
pull_size: 10
filters_1: 6
Score: 0.969565212726593

Trial 05 summary
Hyperparameters:
kernel_size: 3
1st_dense_units: 4
2nd_dense_activation: sigmoid
learning_rate: 0.008002917774687932
filters_0: 4
conv2d_activation: tanh
pull_size: 8
filters_1: 4
Score: 0.9420289993286133

Trial 07 summary
Hyperparameters:
kernel_size: 2
1st_dense_units: 8
2nd_dense_activation: sigmoid
learning_rate: 0.0003233648014907526
filters_0: 4
conv2d_activation: tanh
pull_size: 6
filters_1: 4
Score: 0.9402174055576324

Trial 08 summary
Hyperparameters:
kerne

#### Tune 3 conv2d layers CNN

In [8]:
num_of_layers = 3
tuner_3_layer_cnn = RandomSearch(
    CNNHyperModel(input_shape, num_classes, num_of_layers),
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=2,
    directory="hpo_tuner/cnn",
    project_name="cnn_time_series_3_conv1d_tuning",
)
tuner_3_layer_cnn.search(train_dataset, epochs=5, validation_data=val_dataset)
tuner_3_layer_cnn.results_summary()

Trial 10 Complete [00h 07m 06s]
val_accuracy: 0.968478262424469

Best val_accuracy So Far: 0.9771738946437836
Total elapsed time: 01h 15m 42s
Results summary
Results in hpo_tuner/cnn/cnn_time_series_3_conv1d_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 04 summary
Hyperparameters:
kernel_size: 5
1st_dense_units: 12
2nd_dense_activation: sigmoid
learning_rate: 0.0011775930616089965
filters_0: 4
conv2d_activation: tanh
pull_size: 8
filters_1: 4
filters_2: 8
Score: 0.9771738946437836

Trial 05 summary
Hyperparameters:
kernel_size: 5
1st_dense_units: 8
2nd_dense_activation: softmax
learning_rate: 0.0011016692761180362
filters_0: 2
conv2d_activation: relu
pull_size: 10
filters_1: 6
filters_2: 6
Score: 0.9702898561954498

Trial 09 summary
Hyperparameters:
kernel_size: 3
1st_dense_units: 32
2nd_dense_activation: softmax
learning_rate: 0.0071544516667882965
filters_0: 2
conv2d_activation: tanh
pull_size: 6
filters_1: 4
filters_2: 2
Score: 0.96847826242446

In [9]:
tuner_1_layer_cnn.search_space_summary()

Search space summary
Default search space size: 7
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3, 5], 'ordered': True}
1st_dense_units (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 32, 'step': 4, 'sampling': 'linear'}
2nd_dense_activation (Choice)
{'default': 'softmax', 'conditions': [], 'values': ['softmax', 'sigmoid'], 'ordered': False}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
filters_0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 2, 'sampling': 'linear'}
conv2d_activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
pull_size (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 10, 'step': 2, 'sampling': 'linear'}


In [10]:
tuner_2_layer_cnn.search_space_summary()

Search space summary
Default search space size: 8
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3, 5], 'ordered': True}
1st_dense_units (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 32, 'step': 4, 'sampling': 'linear'}
2nd_dense_activation (Choice)
{'default': 'softmax', 'conditions': [], 'values': ['softmax', 'sigmoid'], 'ordered': False}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
filters_0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 2, 'sampling': 'linear'}
conv2d_activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
pull_size (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 10, 'step': 2, 'sampling': 'linear'}
filters_1 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 2, 'sampling': 'linear'}


In [11]:
tuner_3_layer_cnn.search_space_summary()

Search space summary
Default search space size: 9
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3, 5], 'ordered': True}
1st_dense_units (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 32, 'step': 4, 'sampling': 'linear'}
2nd_dense_activation (Choice)
{'default': 'softmax', 'conditions': [], 'values': ['softmax', 'sigmoid'], 'ordered': False}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
filters_0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 2, 'sampling': 'linear'}
conv2d_activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
pull_size (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 10, 'step': 2, 'sampling': 'linear'}
filters_1 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 2, 'sampling': 'linear'}
filters_2 (Int)

Print the best hyperparameters

In [12]:
best_hps = tuner_2_layer_cnn.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
for param in best_hps.values:
    print(f"{param}: {best_hps.get(param)}")

Best Hyperparameters:
kernel_size: 2
1st_dense_units: 16
2nd_dense_activation: softmax
learning_rate: 0.008792598590199204
filters_0: 4
conv2d_activation: tanh
pull_size: 10
filters_1: 6


We will use ... models, that showed good (or best) results in HPO

#### Model 1

In [ ]:
# Trial 01 summary
# Hyperparameters:
# kernel_size: 5
# 1st_dense_units: 20
# 2nd_dense_activation: sigmoid
# learning_rate: 0.00392551860940815
# filters_0: 4
# conv2d_activation: relu
# pull_size: 8
# Score: 0.9615941941738129

In [30]:
model_1 = keras.Sequential(
    [
        keras.layers.Input(shape=(48000, 1)),
        keras.layers.Conv1D(filters=4, kernel_size=5, activation="relu"),
        keras.layers.MaxPooling1D(pool_size=8),
        keras.layers.Flatten(),
        keras.layers.Dense(20, activation="relu"),
        keras.layers.Dense(2, activation="sigmoid"),
    ]
)

model_1.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 47996, 4)          24        
                                                                 
 max_pooling1d_8 (MaxPoolin  (None, 5999, 4)           0         
 g1D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 23996)             0         
                                                                 
 dense_10 (Dense)            (None, 20)                479940    
                                                                 
 dense_11 (Dense)            (None, 2)                 42        
                                                                 
Total params: 480006 (1.83 MB)
Trainable params: 480006 (1.83 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [31]:
model_1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00392551860940815),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


# EPOCHS = 1
# EPOCHS = 20
EPOCHS = 10
history = model_1.fit(train_dataset, epochs=EPOCHS)

Epoch 1/10


/home/polina/newname/.venv/lib/python3.9/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


353/353 [==============================] - 55s 150ms/step - loss: 0.3176 - accuracy: 0.8649
Epoch 2/10
353/353 [==============================] - 55s 153ms/step - loss: 0.2253 - accuracy: 0.9074
Epoch 3/10
353/353 [==============================] - 57s 160ms/step - loss: 0.1674 - accuracy: 0.9381
Epoch 4/10
353/353 [==============================] - 54s 151ms/step - loss: 0.1525 - accuracy: 0.9423
Epoch 5/10
353/353 [==============================] - 55s 152ms/step - loss: 0.1257 - accuracy: 0.9538
Epoch 6/10
353/353 [==============================] - 55s 154ms/step - loss: 0.0947 - accuracy: 0.9653
Epoch 7/10
353/353 [==============================] - 54s 151ms/step - loss: 0.0805 - accuracy: 0.9710
Epoch 8/10
353/353 [==============================] - 54s 150ms/step - loss: 0.0948 - accuracy: 0.9681
Epoch 9/10
353/353 [==============================] - 57s 158ms/step - loss: 0.0777 - accuracy: 0.9740
Epoch 10/10
353/353 [==============================] - 54s 150ms/step - loss: 0.0622

In [34]:
print("Validation dataset accuracy:")
val_loss, val_acc = model_1.evaluate(x_val_np, y_val_np)

Validation dataset accuracy:
44/44 [==============================] - 2s 36ms/step - loss: 0.2669 - accuracy: 0.9435


In [35]:
print("Validation dataset:")
(
    y_pred_val,
    non_overlap_patritions_f1_scores_val,
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(model_1, x_val_np, y_val_np, model_format="keras")

print("\nTest dataset:")
(
    y_pred_test,
    non_overlap_patritions_f1_scores_test,
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(model_1, x_test_np, y_test_np, model_format="keras")

print("Time of one prediction for Test dataset:")
evaluate_time_of_prediction(
    model_1, x_test_np, y_test_np, model_format="keras", show_prediction_evaluation=True
)

Validation dataset:
44/44 [==============================] - 1s 32ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 94.20%
Recall: 87.29%
Precision: 95.37%
F1-score: 91.15%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9109103035957894

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9102927859588559

Test dataset:
44/44 [==============================] - 2s 36ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 94.54%
Recall: 90.61%
Precision: 92.63%
F1-score: 91.61%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9155301305164129

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9137298632059566
Time of one prediction for Test dataset:
Accuracy: 94.54%
Recall: 90.61%


In [36]:
MODEL_FILE_NAME = (
    "time_series_models_from_notebooks/cnn/hpo/cnn_time_ser_1_conv_layer_model.keras"
)
model_1.save(MODEL_FILE_NAME)
print("Model file name: ", MODEL_FILE_NAME)
convert_bytes(get_file_size(MODEL_FILE_NAME), "MB")

Model file name:  spectrogram_models_from_notebooks/cnn/hpo/cnn_time_ser_1_conv_layer_model.keras
File size: 5.522 Megabytes


1 layer CNN is too big. We dont have second MaxPooling

#### Model 2

I trained this model for 5 and for 10 epochs. Less epochs (5) give better F1-score

In [37]:
best_hps = tuner_2_layer_cnn.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
for param in best_hps.values:
    print(f"{param}: {best_hps.get(param)}")

# Trial 05 summary
# Hyperparameters:
# kernel_size: 5
# 1st_dense_units: 8
# 2nd_dense_activation: softmax
# learning_rate: 0.0011016692761180362
# filters_0: 2
# conv2d_activation: relu
# pull_size: 10
# filters_1: 6
# filters_2: 6
# Score: 0.9702898561954498

Best Hyperparameters:
kernel_size: 2
1st_dense_units: 16
2nd_dense_activation: softmax
learning_rate: 0.008792598590199204
filters_0: 4
conv2d_activation: tanh
pull_size: 10
filters_1: 6


Model 10 epochs

In [46]:
model_2_10_epochs = keras.Sequential(
    [
        keras.layers.Input(shape=(48000, 1)),
        keras.layers.Conv1D(filters=4, kernel_size=2, activation="tanh"),
        keras.layers.MaxPooling1D(pool_size=10),
        keras.layers.Conv1D(filters=6, kernel_size=2, activation="tanh"),
        keras.layers.MaxPooling1D(pool_size=10),
        keras.layers.Flatten(),
        keras.layers.Dense(16, activation="relu"),
        keras.layers.Dense(2, activation="softmax"),
    ]
)

model_2_10_epochs.summary()

model_2_10_epochs.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.008792598590199204),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


# EPOCHS = 1
EPOCHS = 10
# EPOCHS = 20
history = model_2_10_epochs.fit(train_dataset, epochs=EPOCHS)

print("Validation dataset accuracy:")
val_loss, val_acc = model_2_10_epochs.evaluate(x_val_np, y_val_np)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_13 (Conv1D)          (None, 47999, 4)          12        
                                                                 
 max_pooling1d_13 (MaxPooli  (None, 4799, 4)           0         
 ng1D)                                                           
                                                                 
 conv1d_14 (Conv1D)          (None, 4798, 6)           54        
                                                                 
 max_pooling1d_14 (MaxPooli  (None, 479, 6)            0         
 ng1D)                                                           
                                                                 
 flatten_8 (Flatten)         (None, 2874)              0         
                                                                 
 dense_16 (Dense)            (None, 16)               

/home/polina/newname/.venv/lib/python3.9/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


353/353 [==============================] - 41s 113ms/step - loss: 0.2954 - accuracy: 0.8604
Epoch 2/10
353/353 [==============================] - 41s 114ms/step - loss: 0.1639 - accuracy: 0.9401
Epoch 3/10
353/353 [==============================] - 50s 139ms/step - loss: 0.1350 - accuracy: 0.9524
Epoch 4/10
353/353 [==============================] - 50s 138ms/step - loss: 0.1053 - accuracy: 0.9643
Epoch 5/10
353/353 [==============================] - 50s 140ms/step - loss: 0.1048 - accuracy: 0.9649
Epoch 6/10
353/353 [==============================] - 48s 133ms/step - loss: 0.0864 - accuracy: 0.9703
Epoch 7/10
353/353 [==============================] - 47s 131ms/step - loss: 0.0715 - accuracy: 0.9762
Epoch 8/10
353/353 [==============================] - 48s 134ms/step - loss: 0.0628 - accuracy: 0.9796
Epoch 9/10
353/353 [==============================] - 48s 134ms/step - loss: 0.0709 - accuracy: 0.9752
Epoch 10/10
353/353 [==============================] - 47s 131ms/step - loss: 0.0644

In [47]:
print("Validation dataset:")
(
    y_pred_val,
    non_overlap_patritions_f1_scores_val,
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(
    model_2_10_epochs, x_val_np, y_val_np, model_format="keras"
)

print("\nTest dataset:")
(
    y_pred_test,
    non_overlap_patritions_f1_scores_test,
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(
    model_2_10_epochs, x_test_np, y_test_np, model_format="keras"
)

print("Time of one prediction for Test dataset:")
evaluate_time_of_prediction(
    model_2_10_epochs,
    x_test_np,
    y_test_np,
    model_format="keras",
    show_prediction_evaluation=True,
)

Validation dataset:
44/44 [==============================] - 1s 27ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.88%
Recall: 98.09%
Precision: 93.16%
F1-score: 95.56%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9559573585383309

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9624568754092924

Test dataset:
44/44 [==============================] - 1s 26ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 95.62%
Recall: 96.94%
Precision: 90.43%
F1-score: 93.57%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9351237580868015

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.938288538065416
Time of one prediction for Test dataset:
Accuracy: 95.62%
Recall: 96.94%
P

In [42]:
model_2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.008792598590199204),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


# EPOCHS = 1
EPOCHS = 5
# EPOCHS = 20
history = model_2.fit(train_dataset, epochs=EPOCHS)

print("Validation dataset accuracy:")
val_loss, val_acc = model_2.evaluate(x_val_np, y_val_np)

Epoch 1/5


353/353 [==============================] - 46s 126ms/step - loss: 0.2764 - accuracy: 0.8801
Epoch 2/5
353/353 [==============================] - 47s 130ms/step - loss: 0.1706 - accuracy: 0.9350
Epoch 3/5
353/353 [==============================] - 47s 129ms/step - loss: 0.1277 - accuracy: 0.9518
Epoch 4/5
353/353 [==============================] - 50s 140ms/step - loss: 0.1080 - accuracy: 0.9602
Epoch 5/5
353/353 [==============================] - 51s 141ms/step - loss: 0.0943 - accuracy: 0.9682
Validation dataset accuracy:
44/44 [==============================] - 2s 30ms/step - loss: 0.1074 - accuracy: 0.9638


In [43]:
print("Validation dataset:")
(
    y_pred_val,
    non_overlap_patritions_f1_scores_val,
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(model_2, x_val_np, y_val_np, model_format="keras")

print("\nTest dataset:")
(
    y_pred_test,
    non_overlap_patritions_f1_scores_test,
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(model_2, x_test_np, y_test_np, model_format="keras")

print("Time of one prediction for Test dataset:")
evaluate_time_of_prediction(
    model_2, x_test_np, y_test_np, model_format="keras", show_prediction_evaluation=True
)

Validation dataset:
44/44 [==============================] - 1s 28ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.38%
Recall: 95.13%
Precision: 94.33%
F1-score: 94.73%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9471310832487327

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9506330911683708

Test dataset:
44/44 [==============================] - 1s 30ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.63%
Recall: 95.85%
Precision: 94.00%
F1-score: 94.92%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9482034905815224

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9468597200306423
Time of one prediction for Test dataset:
Accuracy: 96.63%
Recall: 95.85%


In [44]:
MODEL_FILE_NAME = (
    "time_series_models_from_notebooks/cnn/hpo/cnn_time_ser_2_conv_layer_model.keras"
)
model_2.save(MODEL_FILE_NAME)
print("Model file name: ", MODEL_FILE_NAME)
convert_bytes(get_file_size(MODEL_FILE_NAME), "MB")

Model file name:  spectrogram_models_from_notebooks/cnn/hpo/cnn_time_ser_2_conv_layer_model.keras
File size: 0.563 Megabytes


#### Model 3

In [48]:
best_hps = tuner_3_layer_cnn.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
for param in best_hps.values:
    print(f"{param}: {best_hps.get(param)}")

Best Hyperparameters:
kernel_size: 5
1st_dense_units: 12
2nd_dense_activation: sigmoid
learning_rate: 0.0011775930616089965
filters_0: 4
conv2d_activation: tanh
pull_size: 8
filters_1: 4
filters_2: 8


In [10]:
model_3_5_epochs = keras.Sequential(
    [
        keras.layers.Input(shape=(48000, 1)),
        keras.layers.Conv1D(filters=4, kernel_size=5, activation="tanh"),
        keras.layers.MaxPooling1D(pool_size=8),
        keras.layers.Conv1D(filters=4, kernel_size=5, activation="tanh"),
        keras.layers.MaxPooling1D(pool_size=8),
        keras.layers.Conv1D(filters=8, kernel_size=5, activation="tanh"),
        keras.layers.MaxPooling1D(pool_size=8),
        keras.layers.Flatten(),
        keras.layers.Dense(12, activation="relu"),
        keras.layers.Dense(2, activation="sigmoid"),
    ]
)

model_3_5_epochs.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 47996, 4)          24        
                                                                 
 max_pooling1d (MaxPooling1  (None, 5999, 4)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 5995, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 749, 4)            0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 745, 8)            168       
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 93, 8)             0

In [11]:
model_3_5_epochs.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0011775930616089965),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


# EPOCHS = 1
EPOCHS = 5
# EPOCHS = 10
history = model_3_5_epochs.fit(train_dataset, epochs=EPOCHS)

print("Validation dataset accuracy:")
val_loss, val_acc = model_3_5_epochs.evaluate(x_val_np, y_val_np)

Epoch 1/5


/home/polina/newname/.venv/lib/python3.9/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


353/353 [==============================] - 41s 112ms/step - loss: 0.2602 - accuracy: 0.8858
Epoch 2/5
353/353 [==============================] - 41s 115ms/step - loss: 0.1081 - accuracy: 0.9640
Epoch 3/5
353/353 [==============================] - 43s 121ms/step - loss: 0.0862 - accuracy: 0.9723
Epoch 4/5
353/353 [==============================] - 43s 120ms/step - loss: 0.0679 - accuracy: 0.9782
Epoch 5/5
353/353 [==============================] - 42s 118ms/step - loss: 0.0621 - accuracy: 0.9794
Validation dataset accuracy:
44/44 [==============================] - 1s 26ms/step - loss: 0.0615 - accuracy: 0.9819


In [12]:
print("Validation dataset:")
(
    y_pred_val,
    non_overlap_patritions_f1_scores_val,
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(
    model_3_5_epochs, x_val_np, y_val_np, model_format="keras"
)

print("\nTest dataset:")
(
    y_pred_test,
    non_overlap_patritions_f1_scores_test,
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(
    model_3_5_epochs, x_test_np, y_test_np, model_format="keras"
)

print("Time of one prediction for Test dataset:")
evaluate_time_of_prediction(
    model_3_5_epochs,
    x_test_np,
    y_test_np,
    model_format="keras",
    show_prediction_evaluation=True,
)

Validation dataset:
44/44 [==============================] - 1s 27ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.19%
Recall: 97.25%
Precision: 97.45%
F1-score: 97.35%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9733570239972023

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9749159656636003

Test dataset:
44/44 [==============================] - 1s 28ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.56%
Recall: 97.16%
Precision: 98.45%
F1-score: 97.80%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9775628286814755

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9757424045006187
Time of one prediction for Test dataset:
Accuracy: 98.56%
Recall: 97.16%


In [8]:
model_3_10_epochs = keras.Sequential(
    [
        keras.layers.Input(shape=(48000, 1)),
        keras.layers.Conv1D(filters=4, kernel_size=5, activation="tanh"),
        keras.layers.MaxPooling1D(pool_size=8),
        keras.layers.Conv1D(filters=4, kernel_size=5, activation="tanh"),
        keras.layers.MaxPooling1D(pool_size=8),
        keras.layers.Conv1D(filters=8, kernel_size=5, activation="tanh"),
        keras.layers.MaxPooling1D(pool_size=8),
        keras.layers.Flatten(),
        keras.layers.Dense(12, activation="relu"),
        keras.layers.Dense(2, activation="sigmoid"),
    ]
)

model_3_10_epochs.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 47996, 4)          24        
                                                                 
 max_pooling1d (MaxPooling1  (None, 5999, 4)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 5995, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 749, 4)            0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 745, 8)            168       
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 93, 8)             0

In [9]:
model_3_10_epochs.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0011775930616089965),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


# EPOCHS = 1
EPOCHS = 10
history = model_3_10_epochs.fit(train_dataset, epochs=EPOCHS)

print("Validation dataset accuracy:")
val_loss, val_acc = model_3_10_epochs.evaluate(x_val_np, y_val_np)

Epoch 1/10


/home/polina/newname/.venv/lib/python3.9/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


353/353 [==============================] - 56s 153ms/step - loss: 0.2602 - accuracy: 0.8858
Epoch 2/10
353/353 [==============================] - 53s 148ms/step - loss: 0.1081 - accuracy: 0.9640
Epoch 3/10
353/353 [==============================] - 62s 174ms/step - loss: 0.0862 - accuracy: 0.9723
Epoch 4/10
353/353 [==============================] - 69s 193ms/step - loss: 0.0679 - accuracy: 0.9782
Epoch 5/10
353/353 [==============================] - 75s 208ms/step - loss: 0.0621 - accuracy: 0.9794
Epoch 6/10
353/353 [==============================] - 63s 176ms/step - loss: 0.0495 - accuracy: 0.9824
Epoch 7/10
353/353 [==============================] - 63s 174ms/step - loss: 0.0445 - accuracy: 0.9848
Epoch 8/10
353/353 [==============================] - 62s 172ms/step - loss: 0.0358 - accuracy: 0.9871
Epoch 9/10
353/353 [==============================] - 62s 172ms/step - loss: 0.0310 - accuracy: 0.9893
Epoch 10/10
353/353 [==============================] - 61s 170ms/step - loss: 0.0322

2024-09-22 17:57:43.999459: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 264960000 exceeds 10% of free system memory.


44/44 [==============================] - 2s 39ms/step - loss: 0.0657 - accuracy: 0.9833


In [10]:
print("Validation dataset:")
(
    y_pred_val,
    non_overlap_patritions_f1_scores_val,
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(
    model_3_10_epochs, x_val_np, y_val_np, model_format="keras"
)

print("\nTest dataset:")
(
    y_pred_test,
    non_overlap_patritions_f1_scores_test,
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(
    model_3_10_epochs, x_test_np, y_test_np, model_format="keras"
)

print("Time of one prediction for Test dataset:")
evaluate_time_of_prediction(
    model_3_10_epochs,
    x_test_np,
    y_test_np,
    model_format="keras",
    show_prediction_evaluation=True,
)

Validation dataset:
 1/44 [..............................] - ETA: 6s

2024-09-22 18:01:54.874296: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 264960000 exceeds 10% of free system memory.


44/44 [==============================] - 2s 39ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.33%
Recall: 97.25%
Precision: 97.87%
F1-score: 97.56%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9753312320782165

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9769318421377577

Test dataset:
 1/44 [..............................] - ETA: 3s

2024-09-22 18:02:25.346150: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 267456000 exceeds 10% of free system memory.


44/44 [==============================] - 3s 70ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.85%
Recall: 97.82%
Precision: 98.68%
F1-score: 98.25%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9817988410806853

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9810600944409472
Time of one prediction for Test dataset:
Accuracy: 98.85%
Recall: 97.82%
Precision: 98.68%
F1-score: 98.25%

Time to make a prediction for a single data point
Mean: 0.078 seconds
Max: 0.456 seconds
Min: 0.044 seconds


In [12]:
MODEL_FILE_NAME = "../../models/time_series_models_from_notebooks/cnn/hpo/cnn_time_ser_3_conv_layer_model.keras"
# model_3_10_epochs.save(MODEL_FILE_NAME)
print("Model file name: ", MODEL_FILE_NAME)
convert_bytes(get_file_size(MODEL_FILE_NAME), "MB")

Model file name:  ../../models/time_series_models_from_notebooks/cnn/hpo/cnn_time_ser_3_conv_layer_model.keras
File size: 0.149 Megabytes


#### Final model

3 layers model trained for 10 epochs will be used as final one as it has very good f1-score and really small size

In [13]:
print("Model file name: ", MODEL_FILE_NAME)

Model file name:  ../../models/time_series_models_from_notebooks/cnn/hpo/cnn_time_ser_3_conv_layer_model.keras
